In [1]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordLevel, BPE
from tokenizers.pre_tokenizers import Whitespace,Split,ByteLevel
from tokenizers.normalizers import Lowercase, NFKC
import os
import polars as pl
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import json
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordLevelTrainer, WordPieceTrainer
import gc

from rdkit import Chem
from rdkit.Chem import AllChem
import atomInSmiles
import mapply
from collections import Counter

multiprocessing.cpu_count()

80

In [2]:
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v4.csv').select(
    pl.col('*')
        # pl.col('molecule'),
        # pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
    ).collect()
print(test_df.estimated_size('gb'), 'GB')
test_df

0.12669966649264097 GB


molecule,bb1,bb2,bb3,id_BRD4,id_HSA,id_sEH,mol_group,group_BRD4,group_HSA,group_sEH
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""C#CCCC[C@H](Nc…",1989,409,409,295246830,295246831,295246832,2,6,7,8
"""C#CCCC[C@H](Nc…",1989,409,1012,295246833,295246834,295246835,2,6,7,8
"""C#CCCC[C@H](Nc…",1989,409,1722,295246836,295246837,295246838,2,6,7,8
"""C#CCCC[C@H](Nc…",1989,409,1078,295246839,295246840,295246841,2,6,7,8
"""C#CCCC[C@H](Nc…",1989,409,605,295246842,295246843,295246844,2,6,7,8
…,…,…,…,…,…,…,…,…,…,…
"""Cn1ncc2cc(Nc3n…",141,1699,307,296921711,296921712,296921713,0,0,1,2
"""[N-]=[N+]=NCCC…",141,1699,1254,296921714,296921715,296921716,0,0,1,2
"""COC(=O)c1ccnc(…",141,1415,1390,296921717,296921718,296921719,0,0,1,2


In [3]:
test_df.group_by('mol_group').agg(pl.col('molecule').count())

mol_group,molecule
i64,u32
0,369039
3,11322
1,486390
2,11271


In [4]:
1674896 - 369039 * 3

567779

In [5]:
pred_ids = test_df[['id_BRD4', 'id_HSA', 'id_sEH']].to_numpy().reshape(-1)
group_ids = test_df[['group_BRD4', 'group_HSA', 'group_sEH']].to_numpy().reshape(-1)
mask = (pred_ids != 0)
assert(mask.sum()==1674896)
PRED_IDS = pred_ids[mask]
GROUP_IDS = group_ids[mask]
print(PRED_IDS.shape, GROUP_IDS.shape)

(1674896,) (1674896,)


In [6]:
(GROUP_IDS<3).sum()

1107117

In [7]:
np.unique(GROUP_IDS)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [10]:
SHARE = [0, 1, 2]
NONSHARE = [3, 4, 5, 6, 7, 8, 9, 10, 11]
ALL = SHARE + NONSHARE

In [9]:
def ensemble(
    test_df,
    submissions = [],
    save_path = None,
):
    ret_df = pd.DataFrame({
        'id': PRED_IDS,
        'group': GROUP_IDS,
        'binds': -1.0
    })
    subset_to_binds = {}
    for i, (csv_path, subsets) in enumerate(submissions):
        print(f'Processing {csv_path}')
        binds = pl.scan_csv(csv_path).select(pl.col('binds')).collect().to_numpy()[:, 0]
        for subset in subsets:
            subset_to_binds.setdefault(subset, []).append(binds[GROUP_IDS == subset])

    for subset, subset_binds in subset_to_binds.items():
        print(f'---subset {subset}---')
        subset_binds = np.array(subset_binds)
        print(subset_binds.shape)
        subset_binds = np.mean(subset_binds, axis = 0)
        print('after agg:', subset_binds.shape)
        ret_df.loc[GROUP_IDS == subset, 'binds'] = subset_binds
    assert (ret_df['binds'] >= 0).all()
    ret_df = ret_df[['id', 'binds']].reset_index(drop = True)
    if save_path is not None:
        ret_df.to_csv(save_path, index = False)
    return ret_df

In [11]:
ret_df = ensemble(test_df,
    [
    ('/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/squeezeformer_0.1033_0.6376.csv', ALL),
    ('/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/lb0.41.csv', ALL)
    ],
    '/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/lb041+squeeze.csv'                 
    )
ret_df

Processing /home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/squeezeformer_0.1033_0.6376.csv
Processing /home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/lb0.41.csv
---subset 0---
(2, 369039)
after agg: (369039,)
---subset 1---
(2, 369039)
after agg: (369039,)
---subset 2---
(2, 369039)
after agg: (369039,)
---subset 3---
(2, 167227)
after agg: (167227,)
---subset 4---
(2, 166263)
after agg: (166263,)
---subset 5---
(2, 166510)
after agg: (166510,)
---subset 6---
(2, 11271)
after agg: (11271,)
---subset 7---
(2, 11271)
after agg: (11271,)
---subset 8---
(2, 11271)
after agg: (11271,)
---subset 9---
(2, 11322)
after agg: (11322,)
---subset 10---
(2, 11322)
after agg: (11322,)
---subset 11---
(2, 11322)
after agg: (11322,)


,id,binds
0,295246830,5.096197e-06
1,295246831,8.973479e-05
2,295246832,5.364418e-07
3,295246833,1.943111e-05
4,295246834,8.359551e-05
...,...,...
1674891,296921721,1.475215e-05
1674892,296921722,1.817942e-06
1674893,296921723,2.086163e-07
1674894,296921724,1.534820e-05


In [117]:
df1 = pl.scan_csv('/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/v3.csv').collect()
df2 = pl.scan_csv('/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/lb0.41.csv').collect()
# df2 = pl.scan_csv('/home/dangnh36/projects/competitions/kaggle_leash_belka/src/outputs/ensemble_v2.csv').collect()

In [118]:
(df1['binds'] == df2['binds']).all()

False

In [119]:

(df1['binds'] == df2['binds']).sum()

1112479